### Changing to the main directory

In [1]:
%cd ..

/home/isham/Desktop/machine-learning-projects/fine-tuning-q-and-a


/home/isham/anaconda3/envs/fine-tuning-q-and-a/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Importing Necessary Libraries

In [13]:
from utils import DATASET_ID_OR_URL, DATA_DIR, DATA_PATH, PROCESSED_DATA_DIR
from utils import clean_text
import opendatasets as od
import pandas as pd

import os

### Data Ingestion

In [2]:
od.download(DATASET_ID_OR_URL, DATA_DIR)

Skipping, found downloaded files in "data/nvidia-documentation-question-and-answer-pairs" (use force=True to force download)


In [3]:
dataset = pd.read_csv(DATA_PATH)

In [4]:
dataset.sample(n=5, random_state=42)

,Unnamed: 0,question,answer
4982,4982,What was the purpose of setting up the DGX RAI...,"In version 2 of the pipeline, the DGX RAID mem..."
4450,4450,When will the full programming model enhanceme...,The full programming model enhancements for th...
4513,4513,What impact does the PTX6 memory consistency m...,The PTX6 memory consistency model introduced s...
4836,4836,What technologies and framework did the resear...,"The researchers used the MatConvNet framework,..."
3800,3800,What does the source-to-instruction correlatio...,The source-to-instruction correlation view in ...


In [5]:
dataset.drop(columns=["Unnamed: 0"], inplace=True)

### Splitting Dataset into Training, Validation, and Test Sets

In [6]:
from sklearn.model_selection import train_test_split

# Settings for 70 - 15 - 15 split of training, validation, and test datasets
train_df, test_df = train_test_split(dataset, test_size=0.15, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1765, random_state=42)

print(f"Train set shape: {train_df.shape}")
print(f"Validation set shape: {val_df.shape}")
print(f"Test set shape: {test_df.shape}")

Train set shape: (4974, 2)
Validation set shape: (1067, 2)
Test set shape: (1067, 2)


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4974 entries, 4221 to 3305
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  4974 non-null   object
 1   answer    4974 non-null   object
dtypes: object(2)
memory usage: 116.6+ KB


In [8]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067 entries, 5559 to 1366
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  1067 non-null   object
 1   answer    1067 non-null   object
dtypes: object(2)
memory usage: 25.0+ KB


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067 entries, 4982 to 265
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  1067 non-null   object
 1   answer    1067 non-null   object
dtypes: object(2)
memory usage: 25.0+ KB


There are no missing values in the dataset which indicates its well curated.

### Cleaning the Dataset

In [10]:
train_df['question'] = train_df['question'].apply(clean_text)
val_df['question'] = val_df['question'].apply(clean_text)
test_df['question'] = test_df['question'].apply(clean_text)

train_df['answer'] = train_df['answer'].apply(clean_text)
val_df['answer'] = val_df['answer'].apply(clean_text)
test_df['answer'] = test_df['answer'].apply(clean_text)

### Saving the Dataset

In [14]:
train_df.to_csv(os.path.join(PROCESSED_DATA_DIR, "train.csv"), index=False)
val_df.to_csv(os.path.join(PROCESSED_DATA_DIR, "val.csv"), index=False)
test_df.to_csv(os.path.join(PROCESSED_DATA_DIR, "test.csv"), index=False)